# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [32]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [33]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [34]:
train_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [35]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0, 30)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = imresize(image[:,:,0],[100,100])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = imresize(image[:,:,1],[100,100])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = imresize(image[:,:,2],[100,100])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = imresize(image[:,:,0],[100,100])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = imresize(image[:,:,1],[100,100])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = imresize(image[:,:,2],[100,100])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels           

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [36]:
curr_dt_time = datetime.datetime.now()
train_path = '/mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/train'
val_path = '/mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [37]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# Input values
x = 30
y = 100
z = 100

# Input shape
input_shape=(x,y,z,3)

#Define model 1
#In this model we are using 3 D CNN stack
Conv3d_model = Sequential()
Conv3d_model.add(Conv3D(8, kernel_size=(3,3,3), input_shape=input_shape, padding='same'))
Conv3d_model.add(BatchNormalization())
Conv3d_model.add(Activation('relu'))
Conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))

Conv3d_model.add(Conv3D(16, kernel_size=(3,3,3), padding='same'))
Conv3d_model.add(BatchNormalization())
Conv3d_model.add(Activation('relu'))
Conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))

Conv3d_model.add(Conv3D(32, kernel_size=(1,3,3), padding='same'))
Conv3d_model.add(BatchNormalization())
Conv3d_model.add(Activation('relu'))
Conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))

Conv3d_model.add(Conv3D(64, kernel_size=(1,3,3), padding='same'))
Conv3d_model.add(Activation('relu'))
Conv3d_model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
Conv3d_model.add(Flatten())
Conv3d_model.add(Dense(256, activation='relu'))
Conv3d_model.add(Dense(128, activation='relu'))

#softmax layer
Conv3d_model.add(Dense(5, activation='softmax'))


#Define model 2
#In this model we are using CNN-RNN stack
GRU_model = Sequential()
GRU_model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2), activation='relu', padding='same'), input_shape=input_shape))
GRU_model.add(TimeDistributed(Conv2D(32, (3,3), kernel_initializer="he_normal", activation='relu')))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        
GRU_model.add(TimeDistributed(Conv2D(64, (3,3), padding='same', activation='relu')))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        
GRU_model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        
GRU_model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        
GRU_model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
GRU_model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

#Flatten Layers        
GRU_model.add(TimeDistributed(Flatten()))

#RNN Layer
GRU_model.add(GRU(256, return_sequences=False))
#softmax layer
GRU_model.add(Dense(5, activation='softmax'))


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [38]:
optimiser = optimizers.Adam()
Conv3d_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (Conv3d_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 100, 100, 8)   656       
_________________________________________________________________
batch_normalization_10 (Batc (None, 30, 100, 100, 8)   32        
_________________________________________________________________
activation_13 (Activation)   (None, 30, 100, 100, 8)   0         
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 15, 50, 50, 8)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 15, 50, 50, 16)    3472      
_________________________________________________________________
batch_normalization_11 (Batc (None, 15, 50, 50, 16)    64        
_________________________________________________________________
activation_14 (Activation)   (None, 15, 50, 50, 16)    0         
__________

In [39]:
optimiser = optimizers.Adam()
GRU_model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (GRU_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_25 (TimeDis (None, 30, 50, 50, 32)    4736      
_________________________________________________________________
time_distributed_26 (TimeDis (None, 30, 48, 48, 32)    9248      
_________________________________________________________________
time_distributed_27 (TimeDis (None, 30, 24, 24, 32)    0         
_________________________________________________________________
time_distributed_28 (TimeDis (None, 30, 24, 24, 64)    18496     
_________________________________________________________________
time_distributed_29 (TimeDis (None, 30, 12, 12, 64)    0         
_________________________________________________________________
time_distributed_30 (TimeDis (None, 30, 12, 12, 128)   73856     
_________________________________________________________________
time_distributed_31 (TimeDis (None, 30, 6, 6, 128)     0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [41]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR =  ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [14]:
Conv3d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/val ; batch size = Source path =  /mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/train ; batch size = 10
Epoch 1/10
10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 6s - loss: 1.4098 - categorical_accuracy: 0.3923

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 210s 3s/step - loss: 1.4039 - categorical_accuracy: 0.3995 - val_loss: 1.5432 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2019-03-1712_28_10.484679/model-00001-1.40459-0.39668-1.54323-0.38000.h5
Epoch 2/10
67/67 [==============================] - 30s 442ms/step - loss: 1.1996 - categorical_accuracy: 0.4627 - val_loss: 1.9064 - val_categorical_accuracy: 0.4000

Epoch 00002: saving model to model_init_2019-03-1712_28_10.484679/model-00002-1.19961-0.46269-1.90638-0.40000.h5
Epoch 3/10
67/67 [==============================] - 31s 462ms/step - loss: 1.2219 - categorical_accuracy: 0.4428 - val_loss: 3.8356 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2019-03-1712_28_10.484679/model-00003-1.22194-0.44279-3.83560-0.30000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/10
67/67 [==============================] - 31s 463ms/step - loss: 0.9197 - categ

In [15]:
GRU_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 35s 516ms/step - loss: 1.7114 - categorical_accuracy: 0.2189 - val_loss: 1.6878 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to model_init_2019-03-1712_28_10.484679/model-00001-1.71144-0.21891-1.68778-0.18000.h5
Epoch 2/10
67/67 [==============================] - 32s 479ms/step - loss: 1.6676 - categorical_accuracy: 0.1741 - val_loss: 1.6326 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to model_init_2019-03-1712_28_10.484679/model-00002-1.66758-0.17413-1.63259-0.18000.h5
Epoch 3/10
67/67 [==============================] - 33s 487ms/step - loss: 1.6673 - categorical_accuracy: 0.1891 - val_loss: 1.6418 - val_categorical_accuracy: 0.2000

Epoch 00003: saving model to model_init_2019-03-1712_28_10.484679/model-00003-1.66729-0.18905-1.64178-0.20000.h5
Epoch 4/10
67/67 [==============================] - 31s 462ms/step - loss: 1.6913 - categorical_accuracy: 0.1741 - val_loss: 1.6101 - val_categorical_accuracy: 0.2300



In [16]:
batch_size = 10
num_epochs = 20

Conv3d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 31s 467ms/step - loss: 0.4108 - categorical_accuracy: 0.8159 - val_loss: 0.8024 - val_categorical_accuracy: 0.6900

Epoch 00001: saving model to model_init_2019-03-1712_28_10.484679/model-00001-0.41078-0.81592-0.80243-0.69000.h5
Epoch 2/20
67/67 [==============================] - 30s 454ms/step - loss: 0.4542 - categorical_accuracy: 0.8358 - val_loss: 0.7525 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2019-03-1712_28_10.484679/model-00002-0.45418-0.83582-0.75246-0.76000.h5
Epoch 3/20
67/67 [==============================] - 31s 459ms/step - loss: 0.3866 - categorical_accuracy: 0.8607 - val_loss: 1.0043 - val_categorical_accuracy: 0.7100

Epoch 00003: saving model to model_init_2019-03-1712_28_10.484679/model-00003-0.38661-0.86070-1.00431-0.71000.h5
Epoch 4/20
67/67 [==============================] - 29s 439ms/step - loss: 0.3366 - categorical_accuracy: 0.8657 - val_loss: 0.6965 - val_categorical_accuracy: 0.7500



In [ ]:
batch_size = 50
num_epochs = 20

Conv3d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
batch_size = 90
num_epochs = 20

Conv3d_model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/val ; batch size = 10
Source path =  /mnt/disks/user/project/PROJECT/gdown.pl-master/Project_data/train ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.4511 - categorical_accuracy: 0.3723

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:39: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 92s 1s/step - loss: 1.4585 - categorical_accuracy: 0.3672 - val_loss: 1.3379 - val_categorical_accuracy: 0.4700

Epoch 00001: saving model to model_init_2019-03-1714_02_23.462429/model-00001-1.45538-0.37104-1.33790-0.47000.h5
Epoch 2/20
67/67 [==============================] - 29s 439ms/step - loss: 1.1334 - categorical_accuracy: 0.5224 - val_loss: 3.6514 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2019-03-1714_02_23.462429/model-00002-1.13342-0.52239-3.65139-0.25000.h5
Epoch 3/20
67/67 [==============================] - 31s 462ms/step - loss: 1.0939 - categorical_accuracy: 0.5572 - val_loss: 1.0210 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_init_2019-03-1714_02_23.462429/model-00003-1.09389-0.55721-1.02102-0.58000.h5
Epoch 4/20
67/67 [==============================] - 31s 464ms/step - loss: 0.9991 - categorical_accuracy: 0.6418 - val_loss: 1.0994 - val_categorical_accuracy: 0.5300

Epo

### So final model is Convolution 3 D model which gives categorical_accuracy: 0.9950 & val_categorical_accuracy: 0.7900 with batch_size:90 and epoch:20